## Practice Lab 2: Decision tree from scratch (well, we hope!)

### Part 1
- create random list of 5 integers
 - create a function that will find the split values by
     - sorting the list from lowest to highest value
     - finding the midpoint for each consecutive pair of values
     - returning midpoints as a list
 - create new random list of 5 integers and test your split values function 

> wherever you see `list` you can always substitute with `ndarray` if more appropriate

In [5]:
import numpy as np
from numpy.random import default_rng

rnd_list = default_rng(123)
int_list = rnd_list.integers(low=5, high=50, size=5)
int_list

array([ 5, 35, 31,  7, 45], dtype=int64)

In [6]:
def splits(x):
    split_ponit_list = []
    x_sorted = sorted(x)
    for i in range(len(x)-1):
        mid_point = (x_sorted[i]+x_sorted[i+1])/2
        split_ponit_list.append(mid_point)
    return split_ponit_list
                      
splits(int_list)         

[6.0, 19.0, 33.0, 40.0]

### Part 2
 - create random array of integers in shape (5, 3)
 - create a function that iterates over the columns and calculates the split points for each column
 - return as a dictionary with column name as key and list of split points as value
 - verify by hand that you are getting the correct split points

In [7]:
int_twoDlist = rnd_list.integers(low=5, high=100, size=(5,3))
int_twoDlist 

array([[25, 29, 22],
       [36, 21, 38],
       [82, 47, 92],
       [47, 31, 79],
       [82, 86, 89]], dtype=int64)

In [8]:
def splits_per_col(data):
    col_split_pts = {}
    ncols = data.shape[1]
    for c in range(ncols):
        split_pts = splits(data[:,c])
        col_split_pts['col_'+ str(c)] = split_pts
    return col_split_pts

In [9]:
splits_per_col(int_twoDlist)

{'col_0': [30.5, 41.5, 64.5, 82.0],
 'col_1': [25.0, 30.0, 39.0, 66.5],
 'col_2': [30.0, 58.5, 84.0, 90.5]}

### Part 3
 - create random array of integers in shape (5, 2)
 - use the first column as feature data X and the second column as regression target y
 - create a function that takes in this array and
     - calculates the MAE of the root node
     - calculates the split points
     - for each split point
         - split the data into two parts
         - calculates the weighted MAE after the split
     - returns the split point (and the new MAE) that reduces the MAE of the root node the most
     
> Note: this function could use other funtions that perform the individual steps

In [10]:
int_twoDlist = rnd_list.integers(low=5, high=100, size=(5,2))
int_twoDlist

array([[ 7, 53],
       [30, 28],
       [27, 83],
       [80, 25],
       [43, 75]], dtype=int64)

In [11]:
def split_data(x, y, split_pt):
    mask = x>split_pt
    anti_mask = x< split_pt
    x_true = x[mask]
    y_true = y[mask]
    x_false = x[anti_mask]
    y_false = y[anti_mask]
    return x_true, y_true, x_false, y_false

In [16]:
x_true, y_true, x_false, y_false = split_data(int_twoDlist[:,0], int_twoDlist[:, 1],70)

In [19]:
def MAE(y, y_true, y_false):
    y_true_hat = np.mean(y_true)
    y_false_hat = np.mean(y_false)
    y_hat= np.mean(y)
    
    old_mae = np.mean(np.absolute(y-y_hat))
    new_mae = len(y_true)/len(y)*np.mean(np.absolute(y_true-y_true_hat)) +\
              len(y_false)/len(y)*np.mean(np.absolute(y_false-y_false_hat))
    return old_mae, new_mae

In [20]:
MAE(int_twoDlist[:,1],y_true,y_false)

(21.04, 15.4)

### Part 4 (Optional)
 - Modify **Part 3** code so that it can handle more than 1 feature

In [25]:
def get_best_split(data):
    x = data[:,0]
    y = data[:,1]
    split_pts = splits(x)
    results = {}
    for point in split_pts:
        x_true, y_true, x_false, y_false = split_data(x, y, point)
        errors = MAE(y, y_true, y_false)
        results[point] = errors
    best_point = min(results, key=results.get)
    return results, best_point

In [26]:
get_best_split(int_twoDlist)

({17.0: (21.04, 21.0),
  28.5: (21.04, 18.93333333333333),
  36.5: (21.04, 21.333333333333336),
  61.5: (21.04, 15.4)},
 61.5)